In [57]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch as pyt
import torch.nn as nn

In [58]:
#Importing the data and seprating the features and dependent variables
#Feature = X, dependent variables = y
trainset = pd.read_csv('../Data/train.csv')
trainlabels = trainset.pop("WY..Tot.Consump.MMBtu")

testset = pd.read_csv('../Data/test.csv')
testlabels = testset.pop("WY..Tot.Consump.MMBtu")


In [59]:
display(testset)

,Month,Date,Year,NG.Consump.MMBtu,Coal.Consump.MMBtu,Tot.Consump.MMBtu,X...Per.kwh.Mo.Albany,Albany,Big.Horn,Campbell,...,ELEC.GEN.PEL-WY-99.M,ELEC.GEN.PC-WY-99.M,ELEC.GEN.NG-WY-99.M,ELEC.GEN.OOG-WY-99.M,ELEC.GEN.NUC-WY-99.M,ELEC.GEN.HYC-WY-99.M,ELEC.GEN.AOR-WY-99.M,ELEC.GEN.HPS-WY-99.M,ELEC.GEN.OTH-WY-99.M,ELEC.GEN.TSN-WY-99.M
0,8,1,2017,1,45,46,14.51,61.3,70.7,68.9,...,2,0,70,21,0,106,265,0,0,0
1,9,1,2017,1,40,41,14.51,54.2,59.3,59.3,...,2,0,61,28,0,84,275,0,5,0
2,10,1,2017,1,37,38,14.51,39.9,45.3,47.3,...,2,0,66,36,0,80,416,0,6,0
3,11,1,2017,1,35,36,14.51,36.9,33.8,37.8,...,2,0,71,34,0,87,398,0,6,0
4,12,1,2017,1,39,40,14.51,24.7,20.0,26.2,...,6,0,80,38,0,86,422,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,2,1,2023,2,24,26,15.53,19.6,16.5,25.1,...,5,0,163,30,0,45,999,0,7,12
67,3,1,2023,2,25,27,15.53,24.1,28.2,28.0,...,4,0,166,34,0,49,926,0,7,18
68,4,1,2023,2,20,22,15.53,34.6,43.6,40.5,...,4,0,180,31,0,45,878,0,6,22
69,5,1,2023,3,22,25,15.53,51.0,61.0,57.2,...,7,0,180,30,0,93,623,0,1,19


In [68]:
def to_tensor(df): return pyt.tensor(df.values.astype(np.float32))

traininput, traintarget = to_tensor(trainset), to_tensor(trainlabels)

#Neural Net Class Definition
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.l1=nn.Linear(54,80)
        self.l2=nn.Linear(80,1)
        
        self.layer_in = None
        self.act_1 = None
        self.act_2 = None
        self.layer_out = None
        
    def forward(self, x):
        self.layer_in = self.l1(x)
        self.act_1 = pyt.tanh(self.layer_in)
        self.layer_out = self.l2(self.act_1)
        self.act_2 = pyt.sigmoid(self.layer_out)
        return self.act_2
    
net = Net()
#not currently loaded or used, but planned as a different loss function than current
loss_fn = nn.BCELoss()

error = 0

optimizer = pyt.optim.SGD(net.parameters(), lr=0.005/traininput.shape[0])
num_epochs = 60
for epoch in range (num_epochs):
    #begin with zero gradient
    optimizer.zero_grad()
    
    #calculate the network output and sum of squared loss for each datapoint
    output = net(traininput)
    loss = ((output.squeeze() - traintarget.squeeze())**2).sum()
    
    #Calculate gradients and take a descent step
    loss.backward()
    optimizer.step()
    
    #monitor optimization
    error = error+loss
    if epoch % (num_epochs/5) == 0: print(loss, error)

#print(output.squeeze()[:10])
#print(output.squeeze()[:10].round())
#print(traintarget[:10].squeeze())
#plt.plot(output.squeeze().detach().numpy(), 'o')
#plt.plot(traininput.squeeze().detach().numpy(), 'o')

#make the test data into tensors to plug into the model
testinput, testtarget = to_tensor(trainset), to_tensor(trainlabels)
#get a test tensor output from of our test input
test_pred = net(testinput)

#check accuracy
acc = (abs(test_pred.round() - testtarget)).float().mean()
print(float(acc))



tensor(282745.6250, grad_fn=<SumBackward0>) tensor(282745.6250, grad_fn=<AddBackward0>)
tensor(272913.9062, grad_fn=<SumBackward0>) tensor(3557859.5000, grad_fn=<AddBackward0>)
tensor(272904.2500, grad_fn=<SumBackward0>) tensor(6832755., grad_fn=<AddBackward0>)
tensor(272899.8750, grad_fn=<SumBackward0>) tensor(10107575., grad_fn=<AddBackward0>)
tensor(272897.3750, grad_fn=<SumBackward0>) tensor(13382356., grad_fn=<AddBackward0>)
36.6078987121582


In [61]:
def assess_errors(data, labels):
    for i in range(data.shape[0]):
        error[i] = labels[i] - data[i] 
